Merge Datasates for Geolocation Analysis

liberaries

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipaddress
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder




sys.path.append(os.path.abspath('../scripts'))
sys.path.append(os.path.abspath('..src'))

load data

In [2]:
fraud_data = pd.read_csv(r'C:\Users\befekadum\Documents\10x acadamy\week8and9\fraud_detection_system\data\Fraud_Data.csv')
credit_card_data = pd.read_csv(r'C:\Users\befekadum\Documents\10x acadamy\week8and9\fraud_detection_system\data\creditcard.csv')
ip_data = pd.read_csv(r'C:\Users\befekadum\Documents\10x acadamy\week8and9\fraud_detection_system\data\IpAddress_to_Country.csv')

In [3]:
ip_data

,lower_bound_ip_address,upper_bound_ip_address,country
0,1.677722e+07,16777471,Australia
1,1.677747e+07,16777727,China
2,1.677773e+07,16778239,China
3,1.677824e+07,16779263,Australia
4,1.677926e+07,16781311,China
...,...,...,...
138841,3.758092e+09,3758093311,Hong Kong
138842,3.758093e+09,3758094335,India
138843,3.758095e+09,3758095871,China
138844,3.758096e+09,3758096127,Singapore


Convert Ip addresses to integer format

In [4]:
# Convert IP address to integer format (removing the scientific notation)
fraud_data['ip_address'] = fraud_data['ip_address'].astype(int)


In [5]:
# Convert IP ranges in IP-to-Country dataset to integers
ip_data['lower_bound_ip_address'] = ip_data['lower_bound_ip_address'].astype(int)
ip_data['upper_bound_ip_address'] = ip_data['upper_bound_ip_address'].astype(int)


In [6]:
ip_data

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216,16777471,Australia
1,16777472,16777727,China
2,16777728,16778239,China
3,16778240,16779263,Australia
4,16779264,16781311,China
...,...,...,...
138841,3758092288,3758093311,Hong Kong
138842,3758093312,3758094335,India
138843,3758095360,3758095871,China
138844,3758095872,3758096127,Singapore


In [7]:

# Step 1: Create an interval index for the IP ranges in the IP-to-Country dataset
ip_ranges = pd.IntervalIndex.from_arrays(ip_data['lower_bound_ip_address'], 
                                         ip_data['upper_bound_ip_address'], 
                                         closed='both')  # Closed interval to include boundaries

# Step 2: Create a function to map the IP address to the corresponding country
def find_country(ip_address):
    # Find the index of the interval containing the IP address
    idx = ip_ranges.get_indexer([ip_address])
    
    # Return the country if a match is found, otherwise return NaN
    return ip_data['country'].iloc[idx[0]] if idx[0] != -1 else pd.NA

# Step 3: Apply this function to each IP address in the fraud dataset
fraud_data['country'] = fraud_data['ip_address'].apply(find_country)




In [8]:
# Check the merged dataframe
fraud_data

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,732758368,0,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,350311387,0,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2621473820,1,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3840542443,0,<NA>
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,415583117,0,United States
...,...,...,...,...,...,...,...,...,...,...,...,...
151107,345170,2015-01-27 03:03:34,2015-03-29 00:30:47,43,XPSKTWGPWINLR,SEO,Chrome,M,28,3451154526,1,United States
151108,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,LYSFABUCPCGBA,SEO,Safari,M,32,2439047221,0,Netherlands
151109,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,MEQHCSJUBRBFE,SEO,IE,F,26,2748470523,0,Japan
151110,207709,2015-07-09 20:06:07,2015-09-07 09:34:46,46,CMCXFGRHYSTVJ,SEO,Chrome,M,37,3601174708,0,United States


Feature Engineering


In [9]:
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])


Ensure purchase_time is in datetime format

In [10]:
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])



the total number of transactions per user as 

In [11]:
fraud_data['transaction_frequency'] = fraud_data.groupby('user_id')['purchase_time'].transform('count')

# result
fraud_data[['user_id', 'purchase_time', 'transaction_frequency']].head()

,user_id,purchase_time,transaction_frequency
0,22058,2015-04-18 02:47:11,1
1,333320,2015-06-08 01:38:54,1
2,1359,2015-01-01 18:52:45,1
3,150084,2015-05-04 13:54:50,1
4,221365,2015-09-09 18:40:53,1


Transaction Velocity (time between consecutive transactions)

Sort by user_id and purchase_time to get consecutive transactions in the right order

In [12]:
fraud_data = fraud_data.sort_values(by=['user_id', 'purchase_time'])






the time difference between consecutive transactions (in hours)  is calculated as

In [13]:
fraud_data['transaction_velocity'] = fraud_data.groupby('user_id')['purchase_time'].diff().dt.total_seconds() / 3600

# Fill NaN values (first transaction for each user) with a large value or zero
# We can use -1 to indicate no previous transaction
fraud_data.fillna({'transaction_velocity':-1}, inplace=True)


In [14]:
# Check the result
fraud_data[['user_id', 'purchase_time', 'transaction_velocity']].head()

,user_id,purchase_time,transaction_velocity
116708,2,2015-02-21 10:03:37,-1.0
15108,4,2015-09-26 21:32:16,-1.0
46047,8,2015-08-13 11:53:07,-1.0
67650,9,2015-05-20 23:06:42,-1.0
109067,12,2015-03-04 20:56:37,-1.0


In [15]:
fraud_data

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity
116708,2,2015-01-11 03:47:13,2015-02-21 10:03:37,54,FGBQNDNBETFJJ,SEO,Chrome,F,25,880217484,0,United States,1,-1.0
15108,4,2015-06-02 16:40:57,2015-09-26 21:32:16,41,MKFUIVOHLJBYN,Direct,Safari,F,38,2785906106,0,Switzerland,1,-1.0
46047,8,2015-05-28 07:53:06,2015-08-13 11:53:07,47,SCQGQALXBUQZJ,SEO,Chrome,M,25,356056736,0,United States,1,-1.0
67650,9,2015-05-16 15:58:32,2015-05-20 23:06:42,62,IEZOHXPZBIRTE,SEO,FireFox,M,21,759104706,0,<NA>,1,-1.0
109067,12,2015-01-10 06:25:12,2015-03-04 20:56:37,35,MSNWCFEHKTIOY,Ads,Safari,M,19,2985180352,0,Mexico,1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143144,399992,2015-06-04 17:43:28,2015-09-30 07:19:17,82,CNVFEDSCAWXGS,Direct,Safari,M,59,1549469282,0,France,1,-1.0
103577,399993,2015-07-30 03:41:08,2015-10-01 07:16:25,20,XJMTBAKOHKTGR,Direct,IE,F,37,2755338335,0,Germany,1,-1.0
6412,399995,2015-03-10 05:19:31,2015-06-05 23:36:56,33,RYXNXTOXIYRQG,Direct,FireFox,F,35,3701447303,0,Japan,1,-1.0
139579,399997,2015-06-30 09:02:16,2015-10-07 20:56:54,66,XFNGDCCMMBTCB,Direct,Chrome,M,48,3454704355,0,United States,1,-1.0


Normalization and Scaling


I will use StandardScaler to scale continuous features, as it centers the data around the mean and scales it based on standard deviation. This ensures that all features contribute equally to the model.

Features to Scale:
1.purchase_value: Monetary values can vary significantly and need to be scaled.
2.age: Age might not be a very large range, but scaling ensures it remains consistent with other features.
3.transaction_frequency and transaction_velocity: These derived features also need scaling to ensure uniform contribution to the model.

numerical columns for scaling

In [16]:
numerical_cols = ['purchase_value']




In [17]:
scaler = StandardScaler()



Apply scaling on the numerical columns

In [25]:
fraud_data[numerical_cols] = scaler.fit_transform(fraud_data[numerical_cols])

fraud_data[numerical_cols].head()

,purchase_value
116708,0.931338
15108,0.221836
46047,0.549298
67650,1.367955
109067,-0.105627


In [24]:
fraud_data.fillna({'country':'Unknown'}, inplace=True)
fraud_data


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,browser_encoded,source_encoded,device_id_encoded,sex_encoded,country_encoded
116708,2,2015-01-11 03:47:13,2015-02-21 10:03:37,0.931338,FGBQNDNBETFJJ,SEO,Chrome,F,25,880217484,0,United States,1,-1.0,0,2,27684,0,171
15108,4,2015-06-02 16:40:57,2015-09-26 21:32:16,0.221836,MKFUIVOHLJBYN,Direct,Safari,F,38,2785906106,0,Switzerland,1,-1.0,4,1,65372,0,157
46047,8,2015-05-28 07:53:06,2015-08-13 11:53:07,0.549298,SCQGQALXBUQZJ,SEO,Chrome,M,25,356056736,0,United States,1,-1.0,0,2,96071,1,171
67650,9,2015-05-16 15:58:32,2015-05-20 23:06:42,1.367955,IEZOHXPZBIRTE,SEO,FireFox,M,21,759104706,0,Unknown,1,-1.0,1,2,43174,1,172
109067,12,2015-01-10 06:25:12,2015-03-04 20:56:37,-0.105627,MSNWCFEHKTIOY,Ads,Safari,M,19,2985180352,0,Mexico,1,-1.0,4,0,67065,1,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143144,399992,2015-06-04 17:43:28,2015-09-30 07:19:17,2.459497,CNVFEDSCAWXGS,Direct,Safari,M,59,1549469282,0,France,1,-1.0,4,1,13470,1,60
103577,399993,2015-07-30 03:41:08,2015-10-01 07:16:25,-0.924284,XJMTBAKOHKTGR,Direct,IE,F,37,2755338335,0,Germany,1,-1.0,2,1,124084,0,64
6412,399995,2015-03-10 05:19:31,2015-06-05 23:36:56,-0.214781,RYXNXTOXIYRQG,Direct,FireFox,F,35,3701447303,0,Japan,1,-1.0,1,1,95293,0,84
139579,399997,2015-06-30 09:02:16,2015-10-07 20:56:54,1.586263,XFNGDCCMMBTCB,Direct,Chrome,M,48,3454704355,0,United States,1,-1.0,0,1,123264,1,171


Encode Categorical Features

Machine learning models typically expect numerical input, so categorical variables (like source, browser, device_id, country, and sex) need to be converted into numerical form. For this, we will use One-Hot Encoding, which creates binary columns for each unique category.

In [26]:
label_encoder = LabelEncoder()
fraud_data['country_encoded'] = label_encoder.fit_transform(fraud_data['country'])
fraud_data['browser_encoded'] = label_encoder.fit_transform(fraud_data['browser'])
fraud_data['source_encoded'] = label_encoder.fit_transform(fraud_data['source'])
fraud_data['device_id_encoded'] = label_encoder.fit_transform(fraud_data['device_id'])
fraud_data['sex_encoded'] = label_encoder.fit_transform(fraud_data['sex'])

fraud_data

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,browser_encoded,source_encoded,device_id_encoded,sex_encoded,country_encoded
116708,2,2015-01-11 03:47:13,2015-02-21 10:03:37,0.931338,FGBQNDNBETFJJ,SEO,Chrome,F,25,880217484,0,United States,1,-1.0,0,2,27684,0,171
15108,4,2015-06-02 16:40:57,2015-09-26 21:32:16,0.221836,MKFUIVOHLJBYN,Direct,Safari,F,38,2785906106,0,Switzerland,1,-1.0,4,1,65372,0,157
46047,8,2015-05-28 07:53:06,2015-08-13 11:53:07,0.549298,SCQGQALXBUQZJ,SEO,Chrome,M,25,356056736,0,United States,1,-1.0,0,2,96071,1,171
67650,9,2015-05-16 15:58:32,2015-05-20 23:06:42,1.367955,IEZOHXPZBIRTE,SEO,FireFox,M,21,759104706,0,Unknown,1,-1.0,1,2,43174,1,172
109067,12,2015-01-10 06:25:12,2015-03-04 20:56:37,-0.105627,MSNWCFEHKTIOY,Ads,Safari,M,19,2985180352,0,Mexico,1,-1.0,4,0,67065,1,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143144,399992,2015-06-04 17:43:28,2015-09-30 07:19:17,2.459497,CNVFEDSCAWXGS,Direct,Safari,M,59,1549469282,0,France,1,-1.0,4,1,13470,1,60
103577,399993,2015-07-30 03:41:08,2015-10-01 07:16:25,-0.924284,XJMTBAKOHKTGR,Direct,IE,F,37,2755338335,0,Germany,1,-1.0,2,1,124084,0,64
6412,399995,2015-03-10 05:19:31,2015-06-05 23:36:56,-0.214781,RYXNXTOXIYRQG,Direct,FireFox,F,35,3701447303,0,Japan,1,-1.0,1,1,95293,0,84
139579,399997,2015-06-30 09:02:16,2015-10-07 20:56:54,1.586263,XFNGDCCMMBTCB,Direct,Chrome,M,48,3454704355,0,United States,1,-1.0,0,1,123264,1,171


Save file


In [30]:
fraud_data.to_csv(r'C:\Users\befekadum\Documents\10x acadamy\week8and9\fraud_detection_system\data\encoded_data.csv',index=False)